In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from matplotlib.colors import LinearSegmentedColormap, Normalize
import os
import random
import string
import io
from contextlib import redirect_stdout
import datetime


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_colwidth', None)

root_dir = os.path.dirname(os.getcwd()) + "/"

In [ ]:
# Important variables that determine performance
# When any of these variables are changed, keep track of old variable to revert to if poor performance

sequence_length = 5 # Number of .npy files to look back on during training
prediction_horizon = 5 # Number of .npy files to predict

# Width and height of each square in the grid
square_size = 50

# Required total of dbZ in sample size
required_value_threshold = 2500000

batch_size = 4

early_stopping_patience = 15

reduce_lr_patience = 5

training_epochs = 40

In [ ]:
# Load the .npy files into a list
train_npy_files = sorted(glob(root_dir + 'data/train_npy_files/*.npy'))
test_npy_files = sorted(glob(root_dir + 'data/test_npy_files/*.npy'))
validation_npy_files = sorted(glob(root_dir + 'data/validation_npy_files/*.npy'))

# Define the dimensions of the radar images.  This should never change
height, width = 3500, 7000

In [ ]:
def compute_total_squares(square_size):
    if height % square_size != 0 or width % square_size != 0:
        raise ValueError(f"{square_size} is not a common factor of both {height} and {width}.")

    rows = height // square_size
    columns = width // square_size
    return rows * columns

# Assume npy_files, sequence_length, and prediction_horizon are already defined
num_squares = compute_total_squares(square_size)

def split_single_grid(file, square_size, height, width):
    grid = np.load(file).reshape(height, width, 1)
    squares = [
        grid[i:i+square_size, j:j+square_size]
        for i in range(0, height, square_size)
        for j in range(0, width, square_size)
    ]
    return np.array(squares)

# Function to split all grids sequentially (no parallelization)
def split_all_grids_sequential(npy_files, square_size, height, width):
    num_squares_per_grid = (height // square_size) * (width // square_size)
    num_files = len(npy_files)

    # Preallocate space for all the grids that will be split into squares
    all_squares = np.empty((num_files, num_squares_per_grid, square_size, square_size, 1), dtype=np.float32)

    # Sequentially process each file and store the result
    for idx, file in enumerate(npy_files):
        squares = split_single_grid(file, square_size, height, width)
        all_squares[idx] = squares

    return all_squares


def split_into_x_y_squares(all_squares, sequence_length, prediction_horizon):
    num_files, num_squares, square_size, _, _ = all_squares.shape
    num_samples = (num_files - sequence_length - prediction_horizon + 1) * num_squares

    # Pre-allocate arrays for input (X) and output (y) sequences
    X = np.empty((num_samples, sequence_length, square_size, square_size, 1), dtype=np.float32)
    y = np.empty((num_samples, prediction_horizon, square_size, square_size, 1), dtype=np.float32)

    sample_idx = 0
    for i in range(num_files - sequence_length - prediction_horizon + 1):
        for j in range(num_squares):
            # For each sample, take the sequence for that square over time
            x_sample = all_squares[i:i+sequence_length, j]
            X[sample_idx] = x_sample

            y_sample = all_squares[i+sequence_length:i+sequence_length+prediction_horizon, j]
            y[sample_idx] = y_sample

            sample_idx += 1

    return X, y


# Pre-split all grids into squares sequentially
train_all_squares = split_all_grids_sequential(train_npy_files, square_size=square_size, height=height, width=width)
test_all_squares = split_all_grids_sequential(test_npy_files, square_size=square_size, height=height, width=width)
validation_all_squares = split_all_grids_sequential(validation_npy_files, square_size=square_size, height=height, width=width)

# Split the pre-split squares into sequences for training, testing, and validation
X_train, y_train = split_into_x_y_squares(train_all_squares, sequence_length=sequence_length, prediction_horizon=prediction_horizon)
X_test, y_test = split_into_x_y_squares(test_all_squares, sequence_length=sequence_length, prediction_horizon=prediction_horizon)
X_validation, y_validation = split_into_x_y_squares(validation_all_squares, sequence_length=sequence_length, prediction_horizon=prediction_horizon)

In [ ]:
def remove_duplicates_by_max_value(X, y):
    valid_indices = []
    num_samples = X.shape[0]
    
    for i in range(num_samples):
        # Sum the pixel values for both X[i] and y[i]
        x_sum = np.sum(X[i])
        y_sum = np.sum(y[i])
        
        # If the sum is greater than or equal to the threshold, keep this sample
        if x_sum + y_sum >= required_value_threshold:
            valid_indices.append(i)
    
    # Select only the valid indices for X and y
    X_filtered = X[valid_indices]
    y_filtered = y[valid_indices]
    
    return X_filtered, y_filtered


X_train_unique, y_train_unique = remove_duplicates_by_max_value(X_train, y_train)
X_test_unique, y_test_unique = remove_duplicates_by_max_value(X_test, y_test)
X_validation_unique, y_validation_unique = remove_duplicates_by_max_value(X_validation, y_validation)

In [ ]:
# Normalize the data
X_train_normalized = X_train_unique / 80
y_train_normalized = y_train_unique / 80

X_test_normalized = X_test_unique / 80
y_test_normalized = y_test_unique / 80

X_validation_normalized = X_validation_unique / 80
y_validation_normalized = y_validation_unique / 80

print(X_train_normalized.shape)
print(y_train_normalized.shape)
print(X_test_normalized.shape)
print(y_test_normalized.shape)
print(X_validation_normalized.shape)
print(y_validation_normalized.shape)

In [ ]:
# Define the colors for the radar map, introducing white for values between 0 and 15
colors = [
    (0, 0, 0),         # White for values 0-15 (no precipitation or very light)
    (0, 0.7, 0),       # Green (light precipitation)
    (1, 1, 0),         # Yellow (moderate precipitation)
    (1, 0.65, 0),      # Orange (heavy precipitation)
    (1, 0, 0),         # Red (very heavy precipitation)
    (0.6, 0, 0.6)      # Purple (extreme precipitation)
]

breakpoints = [0.0, .15/1.0, .40/1.0, .60/1.0, .70/1.0, 1.0]

# Create the custom colormap
radar_cmap = LinearSegmentedColormap.from_list('radar', colors, N=80)

# Normalize the data range from 0 to 80
norm = Normalize(vmin=0, vmax=1)

# Construct a figure to visualize the images
x_fig, x_axes = plt.subplots(1, 5, figsize=(20, 14))
y_fig, y_axes = plt.subplots(1, 5, figsize=(20, 14))

# Randomly choose a data example to visualize
data_choice = np.random.choice(range(len(X_train_normalized)), size=1)[0]
lastdata = None  # Initialize lastdata to None before looping

# Plot each of the sequential images for one random data example
for idx, ax in enumerate(x_axes.flat):
    x_thisdata = X_train_normalized[data_choice][idx]

    # Compare the current data to the last one if lastdata is not None
    if lastdata is not None and np.array_equal(x_thisdata, lastdata):
        print(f"Frame x - {idx + 1} is the same as the previous frame.")

    # Display the image with the radar colormap and normalization
    im = ax.imshow(np.squeeze(x_thisdata), cmap=radar_cmap, norm=norm)
    
    # Add title and remove axis
    ax.set_title(f"Frame X - {idx + 1}")
    ax.axis("off")

    # Update lastdata
    lastdata = x_thisdata

# Print information and display the figure
print(f"Displaying frames for example {data_choice}.")

for idx, ax in enumerate(y_axes.flat):
    y_thisdata = y_train_normalized[data_choice][idx]

    # Compare the current data to the last one if lastdata is not None
    if lastdata is not None and np.array_equal(y_thisdata, lastdata):
        print(f"Frame y - {idx + 1} is the same as the previous frame.")

    # Display the image with the radar colormap and normalization
    im = ax.imshow(np.squeeze(y_thisdata), cmap=radar_cmap, norm=norm)
    
    # Add title and remove axis
    ax.set_title(f"Frame y - {idx + 1}")
    ax.axis("off")

    # Update lastdata
    lastdata = y_thisdata

plt.tight_layout()
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, Conv3D, BatchNormalization, Input, Activation
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import AdamW
import subprocess

channels = 1  # Reflectivity is your feature, so 1 channel

# Define the model using an Input layer for the input shape
model = Sequential()

# Add Input Layer
model.add(Input(shape=(sequence_length, square_size, square_size, channels))) 

# First ConvLSTM2D layer with return_sequences=True
model.add(ConvLSTM2D(filters=64, kernel_size=(3, 3), padding='same', return_sequences=True))
model.add(BatchNormalization())
model.add(Activation('relu'))

# Second ConvLSTM2D layer with return_sequences=True to return all frames
model.add(ConvLSTM2D(filters=128, kernel_size=(3, 3), padding='same', return_sequences=True))
model.add(BatchNormalization())
model.add(Activation('relu'))

# Third ConvLSTM2D layer
model.add(ConvLSTM2D(filters=256, kernel_size=(3, 3), padding='same', return_sequences=True))
model.add(BatchNormalization())
model.add(Activation('relu'))

# Replace Conv3D with Conv2D to predict the next frame(s)
model.add(Conv3D(filters=1, kernel_size=(3, 3, 3), activation='linear', padding='same'))

optimizer = AdamW(learning_rate=0.001)

# Compile the model
model.compile(loss='mae', optimizer=optimizer)

# Print model summary
model.summary()

In [ ]:
training_id = ''.join(random.choices(string.ascii_letters + string.digits, k=8))
model_name = f"model_{training_id}.keras"

# model = load_model(root_dir + f"model/{model_name}")
# # Create an Adam optimizer with a custom learning rate
# optimizer = AdamW()  # Set the learning rate here

# # Compile the model with the optimizer
# model.compile(loss='mae', optimizer=optimizer)

In [ ]:
def get_model_memory_usage(batch_size, model):
    features_mem = 0  # Initialize memory for features
    float_bytes = 4.0  # Float32 uses 4 bytes
    
    for layer in model.layers:
        # Use layer.output.shape to get the output shape instead of output_shape
        out_shape = layer.output.shape
        
        # Remove the batch size dimension (out_shape[0]) and None (which represents the batch dimension)
        out_shape = [dim for dim in out_shape if dim is not None]
        
        # Multiply all output shape dimensions to calculate the number of elements per layer
        single_layer_mem = 1
        for s in out_shape:
            single_layer_mem *= s
            
        # Convert to memory (in bytes and MB)
        single_layer_mem_float = single_layer_mem * float_bytes  # Multiply by 4 bytes (float32)
        single_layer_mem_MB = single_layer_mem_float / (1024 ** 2)  # Convert to MB
        
        print(f"Memory for layer {layer.name} with output shape {out_shape} is: {single_layer_mem_MB:.2f} MB")
        
        features_mem += single_layer_mem_MB  # Accumulate total feature memory
    
    # Calculate Parameter memory
    trainable_wts = np.sum([K.count_params(p) for p in model.trainable_weights])
    non_trainable_wts = np.sum([K.count_params(p) for p in model.non_trainable_weights])
    parameter_mem_MB = ((trainable_wts + non_trainable_wts) * float_bytes) / (1024 ** 2)
    
    print("_________________________________________")
    print(f"Memory for features in MB is: {features_mem * batch_size:.2f} MB")
    print(f"Memory for parameters in MB is: {parameter_mem_MB:.2f} MB")

    total_memory_MB = (batch_size * features_mem) + parameter_mem_MB
    total_memory_GB = total_memory_MB / 1024  # Convert to GB
    
    return total_memory_GB

#####################################################################

mem_for_my_model = get_model_memory_usage(batch_size, model)

print("_________________________________________")
print("Minimum memory required to work with this model is: %.2f" %mem_for_my_model, "GB")

In [ ]:
cp = ModelCheckpoint(root_dir + f"model/{training_id}/{model_name}", save_best_only=True)
early_stopping = EarlyStopping(
    monitor="val_loss",
    patience=early_stopping_patience,
    min_delta=0.001,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss", 
    patience=reduce_lr_patience,
    factor=0.5,
    min_lr=1e-7,
    min_delta=0.0001,
    cooldown=2,
    verbose=1
)

# Train the model
history = model.fit(X_train_normalized, y_train_normalized, 
          batch_size=batch_size, 
          epochs=training_epochs, 
          callbacks=[cp, early_stopping, reduce_lr],
          validation_data=(X_validation_normalized, y_validation_normalized))

In [ ]:
# Path to the model saved by ModelCheckpoint
model_path = root_dir + f"model/{training_id}/{model_name}"

val_loss = history.history['val_loss']  # List of validation losses for each epoch
average_val_loss = sum(val_loss) / len(val_loss)  # Compute the average val_loss
num_epochs = len(val_loss)  # Number of epochs is the length of the val_loss list

# Create the message including average val_loss and number of epochs
training_results_message = f"Trained model: {model_name} with avg_val_loss: {average_val_loss:.4f} epochs: {num_epochs}"

In [ ]:
mem_output = io.StringIO()
with redirect_stdout(mem_output):
    mem_for_my_model = get_model_memory_usage(batch_size, model)
mem_output_lines = mem_output.getvalue().splitlines()

current_time = datetime.datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC")

results_lines = [
    "## What I tried new ##",
    "",
    f"Model trained on: {current_time}",
    "",
    training_results_message,
    "",
    "## Variables ##",
    f"sequence_length: {sequence_length}",
    f"prediction_horizon: {prediction_horizon}",
    f"square_size: {square_size}",
    f"batch_size: {batch_size}",
    f"early_stopping_patience: {early_stopping_patience}",
    f"reduce_lr_patience: {reduce_lr_patience}",
    f"training_epochs: {training_epochs}",
    f"required_value_threshold: {required_value_threshold}",
    "",
    "## Data Shape ##",
    f"X_train: {X_train_normalized.shape}",
    f"y_train: {y_train_normalized.shape}",
    f"X_test: {X_test_normalized.shape}",
    f"y_test: {y_test_normalized.shape}",
    f"X_validation: {X_validation_normalized.shape}",
    f"y_validation: {y_validation_normalized.shape}",
    "",
    "## Memory Needed Summary ##",
    f"Memory required for model: {mem_for_my_model:.2f} GB",
    *mem_output_lines,
]

results_filename = f"results_{training_id}.txt"
results_filepath = os.path.join(root_dir, f"model/{training_id}", results_filename)

with open(results_filepath, "w") as f:
    f.write("\n".join(results_lines))

In [ ]:
def show_results(known_frames, should_be_frames):
    # Predict future frames
    predicted_frames = model.predict(np.expand_dims(known_frames, axis=0))
    predicted_frames = np.squeeze(predicted_frames, axis=0)
    
    # Construct a figure for the original and predicted frames (5 rows of 3 columns each)
    fig, axes = plt.subplots(3, 5, figsize=(10, 7))  # 5 rows for original + 5 rows for predicted, 3 columns each
    
    for idx in range(5):
        row, col = divmod(idx, 5)
        ax = axes[row, col]
        ax.imshow(np.squeeze(known_frames[idx]), cmap=radar_cmap, norm=norm)
        ax.set_title(f"Original {idx + 1}")
        ax.axis("off")
        
    for idx in range(5):
        row, col = divmod(idx, 5)
        ax = axes[row + 1, col]  # Offset by 5 rows to separate original and predicted
        ax.imshow(np.squeeze(predicted_frames[idx]), cmap=radar_cmap, norm=norm)
        ax.set_title(f"Predicted {idx + 6}")
        ax.axis("off")

    for idx in range(5):
        row, col = divmod(idx, 5)
        ax = axes[row + 2, col]  # Offset by 5 rows to separate original and predicted
        ax.imshow(np.squeeze(should_be_frames[idx]), cmap=radar_cmap, norm=norm)
        ax.set_title(f"Actual {idx + 6}")
        ax.axis("off")
        
    # Display the figure
    plt.tight_layout()
    plt.show()

In [ ]:
trainChoice = np.random.choice(range(len(X_train_normalized)), size=1)[0]
trainFrames = X_train_normalized[trainChoice]
actual_train_frames = y_train_normalized[trainChoice]
show_results(trainFrames, actual_train_frames)

In [ ]:
valChoice = np.random.choice(range(len(X_validation_normalized)), size=1)[0]
val_frames = X_validation_normalized[valChoice]
val_actual_frames = y_validation_normalized[valChoice]
show_results(val_frames, val_actual_frames)

In [ ]:
testChoice = np.random.choice(range(len(X_test_normalized)), size=1)[0]
test_frames = X_test_normalized[testChoice]
actual_test_frames = y_test_normalized[testChoice]
show_results(test_frames, actual_test_frames)